In [52]:
from wordcloud import WordCloud
from matplotlib import pyplot as plt

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib 
from scipy import optimize
from scipy import stats
plt.style.use("ggplot")
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
#plt.rcParams['font.family'] = 'IPAexGothic'

import warnings
warnings.filterwarnings('ignore') # warningが出ないように設定
pd.set_option("display.max_rows", None) # pandasの表示上限をなくす
pd.set_option("display.max_columns", None) # pandasの表示上限をなくす
import pickle
import gspread
import json
from oauth2client.service_account import ServiceAccountCredentials
import collections
import gensim
import random

model = gensim.models.Word2Vec.load("../latest-ja-word2vec-gensim-model/word2vec.gensim.model")

In [53]:
f = open("../fact_relation_vec_list.binaryfile",'rb')
#f = open("../kansei_relation_vec_list.binaryfile",'rb')
mean_vec_list = pickle.load(f)

mean_vec_list = random.sample(mean_vec_list, int(len(mean_vec_list)/20))
len(mean_vec_list)

3696

In [54]:
# (1) Google Spread Sheetsにアクセス
def connect_gspread(jsonf,key):
    scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name(jsonf, scope)
    gc = gspread.authorize(credentials)
    SPREADSHEET_KEY = key
    worksheet = gc.open_by_key(SPREADSHEET_KEY).sheet1
    return worksheet

In [55]:
def result_vec_list_generator(imput_word_list):
    result_vec_list  = []

    for w in imput_word_list:

        try:
            w_to_v = model.wv[w]

            for mean_vec in mean_vec_list:
                result_vec_plus = w_to_v - mean_vec 
                result_vec_minus =   w_to_v + mean_vec 
    
                result_vec_list.append(result_vec_plus)
                result_vec_list.append(result_vec_minus)
        except:
            print("Error", w)

    return result_vec_list

In [56]:
# 関係ベクトルと類似度が高い単語をn個上位表示
def output_word_list_generator(result_vec_list):
    output_word_list = []
    max_n = 2

    for vec in result_vec_list:
        most_similar = np.array(list(model.wv.most_similar([vec], [], max_n)))
        #output_word_list.extend(most_similar[:,0].tolist())
        
        a = most_similar[:,1]
        for cos in a:
            if float(cos) >= 0.85: #コサイン類似度が0.8以下を除去
                output_word_list.extend(most_similar[:,0].tolist())
                
    return output_word_list

In [57]:
def ggg(list_taple):
    l = []
    for taple in list_taple:
        a = taple[0]
        l.append(a)
    return l

In [58]:
# ここでjsonfile名と2-2で用意したkeyを入力
jsonf = "quickstart-1595337771922-22c2ceb47d3c.json"
spread_sheet_key = "1WKpQ4sxL_IP9Fm6551xDRfhxhDLzmgGAG3FCzDFmamI"
worksheet = connect_gspread(jsonf,spread_sheet_key)

In [59]:
# スプレッドシートのデータを取得するコード
# {"answer":[imput_word_list], ...}

data_dic = {}

for i in range(2, 31): #すプレッソシートの行数
    answers_list = worksheet.row_values(i)
    imput_word_list = answers_list[2:]
    answer_word = answers_list[1]
    
    data_dic[answer_word] = imput_word_list

In [63]:
count = 0

for answer_word, imput_word_list in data_dic.items():
    print(answer_word)
    print(imput_word_list)
    
    result_vec_list = result_vec_list_generator(imput_word_list)
    output_word_list = output_word_list_generator(result_vec_list)
    
    for imput_word in imput_word_list:
        output_word_list = [output_word for output_word in output_word_list if output_word != imput_word]
    
    c = collections.Counter(output_word_list)
    dic = dict(c)
    dic2 = sorted(dic.items(), key=lambda x:x[1], reverse=True)
    l = ggg(dic2[:30])
    print(l)
    
    if answer_word in l: 
        print("成功**************************")
        
        count += 1
    
    print("　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　")
    
print("正答率", count/len(data_dic))

LIXIL
['トステム', 'INAX', '新日軽', 'サンウエーブ工業', '東洋エクステリア', 'YKK', '大建工業']
Error サンウエーブ工業
Error 東洋エクステリア
Error 大建工業
['LIXIL', 'JA共済', 'イー・アクセス', 'アサヒビール', '洲本', 'ストアー', '積水ハウス', '東京ガス', 'テーブルマーク', '北陸鉄道', '京阪電気鉄道', 'トムス・エンタテインメント', '（株）', '三越伊勢丹', '西友', 'パナソニック電工', 'デパ地下', '日立建機', '三菱地所', 'Emerging', '日本触媒', '筑波銀行', 'ワイモバイル', 'SCE', '三井化学', '東京海上', 'ヴァニラウェア', 'バビロニアン・キャッスル・サーガ', '飯田町', '聖子']
成功**************************
　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　
ハラール
['イスラム教', 'イスラム', 'アラビア', 'アジア', 'インドネシア']
['イスラーム', 'スカンディナヴィア', 'ムスリム', 'フェニキア', 'パキスタン', 'ラテンアメリカ', 'スカンジナビア', 'ペルシャ', 'ヒンドゥー', '西アフリカ', 'マレーシア', 'アフリカ', 'インド', 'シリア', '東アジア', '東南アジア', 'バングラデシュ', 'イスラム教徒', 'ユダヤ教', '仏教徒', 'アルメニア', 'ニジェール', 'クレオール', '東アフリカ', '中南米', 'イラン', 'ニューカレドニア', 'ミャンマー', '異教徒', 'アラブ人']
　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　
ダイドードリンコ
['キリンビバレッジ', 'ダイドーブレンド', 'サントリー', '日本たばこ産業', 'JT', 'コカ・コーラ']
Error ダイドーブレンド
['アサヒビール', 'P&G', '日清食品', 'サッポロビール', '資生堂', 'コカコーラ', 'イー・アクセス'

['舞鶴', '横須賀', '出島', '定期観光バス', '那覇', '大湊', '館山', '木更津', '晴海', 'こどもの国', '神戸', '銚子', '下関', '豊橋', '幕張', '明石', '外国人居留地', '萩', '柏崎', '熊本', '平戸', '尼崎', '美里', '矢田', '宗谷', '夕張', '帯広', '釧路', 'バスセンター', '広尾']
　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　
PISA
['OECD', 'Programme', '日本']
['経済協力開発機構', '韓国', 'Experiment', '国内総生産', 'Assessment', 'Development', '人間開発指数', '購買力平価', '世界銀行', 'Testing', 'Problem', 'Operational', 'Program', 'Climate', 'Logo', 'Omega', 'Language', 'Code', 'Management', 'Copyright', 'Pathfinder', 'Alchemy', 'Extension', 'Identification', 'Effective', 'academic', 'meeting', 'Space', '日本国内', 'EU']
　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　
スポーツニッポン
['毎日新聞', '日本経済新聞社', '読売新聞', 'サンケイ', '産経新聞']
['朝日新聞', '産経新聞社', '電通', '日本経済新聞', '東京新聞', '日経', '紀伊国屋書店', '夕刊フジ', '中日新聞', '同紙', 'マインドフルネス', '日経BP', '博報堂', '健康法', '田中康夫', '橋下', '共同通信社', 'ベネッセコーポレーション', '日本テレビ放送網', '朝日新聞社', 'ベネッセ', '丸紅', 'イー・アクセス', 'KDDI', 'まんが', '社説', '報道機関', '松尾', '黒田', '鈴木']
　　　　　　　　　　　　　　　

In [ ]:
"""
for i in range(2, 33): #すプレッソシートの行数
    answers_list = worksheet.row_values(i)
    imput_word_list = answers_list[2:]
    answer_word = answers_list[1]
    print("answer", answer_word)
    print("imput", imput_word_list)
    
    result_vec_list = result_vec_list_generator(imput_word_list)
    output_word_list = output_word_list_generator(result_vec_list)
    
    for imput_word in imput_word_list:
        output_word_list = [output_word for output_word in output_word_list if output_word != imput_word]
    
    c = collections.Counter(output_word_list)
    dic = dict(c)
    dic2 = sorted(dic.items(), key=lambda x:x[1], reverse=True)
    l = ggg(dic2[:30])
    print(l)
    
    if answer_word in l: 
        print("成功**************************")
    
    print("　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　")
"""

In [ ]:
# 複数の単語で類似度の高いものを上から１０個表示する
most_similar = np.array(list(model.wv.most_similar(imput_word_list, [], 10)))[:,0]
most_similar.tolist()